# Load data using coffea

In [1]:
import numpy as np
import awkward as ak

In [2]:
import uproot

In [3]:
from coffea import hist, util
import coffea.processor as processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.util import save

In [4]:
class PhotonSelector(processor.ProcessorABC):
    def __init__(self, isMC=True):
        
        # data is/isn't Monte Carlo
        self.isMC = isMC

        # declare axes
        dataset_axis = hist.Cat("dataset","Dataset")

        photon_pt_axis = hist.Bin("pt","$p_{T}$ [GeV]", 40, 0, 200)
        photon_eta_axis = hist.Bin("eta","$\eta$", 50, -2, 2)
        photon_phi_axis = hist.Bin("phi","$\phi$", 64, -3.2, 3.2)
        photon_reliso_all_axis = hist.Bin("reliso","pfRelIso03_all", 40, 0, 0.2)
        photon_reliso_chg_axis = hist.Bin("reliso","pfRelIso03_chg", 40, 0, 0.2)
        photon_sieie_axis = hist.Bin("sieie","$\sigma_{i\eta i\eta}$", 40, 0, 0.03)
        photon_r9_axis = hist.Bin("r9","R9", 40, 0.2, 1.1)
        photon_hoe_axis = hist.Bin("hoe","H over E", 40, 0, 0.1)
        
        mu_deltar_axis = hist.Bin("deltar","$\Delta R$: photon-muon", 50, 0, 4)
        jet_deltar_axis = hist.Bin("deltar","$\Delta R$: photon-jet", 50, 0, 4)
        
        mvaid_axis = hist.Bin("mvaid", "mvaID", 25,-1,1)

        flavLabel_axis = hist.Bin("flav",None,[0,1,2,14])
        photon_genPartFlav_axis = hist.Bin("flav","genPartFlav",14,0,14)
        
        
        # accumulator object: dictionary storing histograms & counters to be filled
        if self.isMC:
            self._accumulator = processor.dict_accumulator({

                ## column accumulators
                'photon_pt': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_eta': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_phi': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_pfRelIso03_all': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_pfRelIso03_chg': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_sieie': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_r9': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_hoe': processor.column_accumulator(np.ndarray(shape=(0,))),
                'deltaR_photon_lepton': processor.column_accumulator(np.ndarray(shape=(0,))),
                'deltaR_photon_jet': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_mvaid': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_genPartFlav': processor.column_accumulator(np.ndarray(shape=(0,))),
                'file': processor.column_accumulator(np.ndarray(shape=(0,))),
                
                ## histograms for features
                'photon_pt_hist': hist.Hist("Counts", dataset_axis, photon_pt_axis, flavLabel_axis),
                'photon_eta_hist': hist.Hist("Counts", dataset_axis, photon_eta_axis, flavLabel_axis),
                'photon_phi_hist': hist.Hist("Counts", dataset_axis, photon_phi_axis, flavLabel_axis),
                'photon_reliso_all_hist': hist.Hist("Counts", dataset_axis, photon_reliso_all_axis, flavLabel_axis),
                'photon_reliso_chg_hist': hist.Hist("Counts", dataset_axis, photon_reliso_chg_axis, flavLabel_axis),
                'photon_sieie_hist': hist.Hist("Counts", dataset_axis, photon_sieie_axis, flavLabel_axis),
                'photon_r9_hist': hist.Hist("Counts", dataset_axis, photon_r9_axis, flavLabel_axis),
                'photon_hoe_hist': hist.Hist("Counts", dataset_axis, photon_hoe_axis, flavLabel_axis),
                'deltaR_photon_lepton_hist': hist.Hist("Counts", dataset_axis, mu_deltar_axis, flavLabel_axis),
                'deltaR_photon_jet_hist': hist.Hist("Counts", dataset_axis, jet_deltar_axis, flavLabel_axis),
                'photon_mvaid_hist': hist.Hist("Counts", dataset_axis, mvaid_axis, flavLabel_axis),
                'photon_genPartFlav_hist': hist.Hist("Counts", dataset_axis, photon_genPartFlav_axis),
            }
            )
        if self.isMC==0:
            self._accumulator = processor.dict_accumulator({
                
                ## column accumulators
                'photon_pt': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_eta': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_phi': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_pfRelIso03_all': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_pfRelIso03_chg': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_sieie': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_r9': processor.column_accumulator(np.ndarray(shape=(0,))),
                'photon_hoe': processor.column_accumulator(np.ndarray(shape=(0,))),
                'deltaR_photon_lepton': processor.column_accumulator(np.ndarray(shape=(0,))),
                'deltaR_photon_jet': processor.column_accumulator(np.ndarray(shape=(0,))),
                
                ## histograms for features
                'photon_pt_hist': hist.Hist("Counts", photon_pt_axis),
                'photon_eta_hist': hist.Hist("Counts", photon_eta_axis),
                'photon_phi_hist': hist.Hist("Counts", photon_phi_axis),
                'photon_reliso_all_hist': hist.Hist("Counts", photon_reliso_all_axis),
                'photon_reliso_chg_hist': hist.Hist("Counts", photon_reliso_chg_axis),
                'photon_sieie_hist': hist.Hist("Counts", photon_sieie_axis),
                'photon_r9_hist': hist.Hist("Counts", photon_r9_axis),
                'photon_hoe_hist': hist.Hist("Counts", photon_hoe_axis),
                'deltaR_photon_lepton_hist': hist.Hist("Counts", mu_deltar_axis),
                'deltaR_photon_jet_hist': hist.Hist("Counts", jet_deltar_axis),
            }
            )

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        
        output = self.accumulator.identity()
        dataset = events.metadata['dataset']
        
        ######## object & event selection ########
        # muons
        muons=events.Muon
        muonSelectTight = ((muons.pt>30) &
                           (abs(muons.eta)<2.4) &
                           (muons.tightId) &
                           (muons.pfRelIso04_all < 0.15)
                          )
        tightMuons = muons[muonSelectTight]
        
        # jets & b-tagged jets
        jets=events.Jet
        jetSelectTight = ((jets.pt>30) &
                          (abs(jets.eta)<2.4) &
                          (jets.isTight)
                         )
        btaggedJetSelect = (jetSelectTight &
                           (jets.btagDeepB>0.6321)
                          )
        tightJets = jets[jetSelectTight]
        tightBJets = jets[btaggedJetSelect]
        
        # electrons
        electrons=events.Electron
        electronSelectTight = ((electrons.pt> 35) &
                               (abs(electrons.eta)<2.1) &
                               electrons.cutBased>=4 # tight cut-based ID
                              )
        tightEle = electrons[electronSelectTight]
        
        
        # delta R cuts
        phoMu, phoMuDR = events.Photon.nearest(tightMuons,return_metric=True)
        phoMuMask = ak.fill_none(phoMuDR>0.4,True)
        
        phoJet, phoJetDR = events.Photon.nearest(tightJets,return_metric=True)
        phoJetMask = ak.fill_none(phoJetDR>0.4,True)
        
        
        # photons
        photons = events.Photon
        photonSelect= ((photons.pt>20) &
                       (abs(photons.eta) < 1.4442) &
                       (photons.isScEtaEE | photons.isScEtaEB) &
                       (photons.electronVeto) & 
                       np.invert(photons.pixelSeed) &
                       phoMuMask & phoJetMask
                       )
        tightPhotons = photons[photonSelect]
        
        tightPhoMu, tightPhoMuDR = tightPhotons.nearest(tightMuons,return_metric=True)
        tightPhoJet, tightPhoJetDR = tightPhotons.nearest(tightJets,return_metric=True)
        
        
        # events
        trigger = events.HLT.IsoMu24 | events.HLT.IsoTkMu24
        
        eventSelection = (trigger &
                          (ak.num(tightMuons)==1) &
                          (ak.num(tightJets)>=4) & 
                          (ak.num(tightBJets)>=1) &
                          (ak.num(tightEle)==0) &
                          (ak.num(tightPhotons)>=1))

        
        ######## fill histograms: Monte Carlo ########
        if self.isMC:
            flavLabel = ak.to_numpy(ak.flatten(tightPhotons[eventSelection].genPartFlav))
            ## flavLabel[flavLabel == 13] = 0 # relabel prompt electrons as fake photons
            
            output['photon_pt_hist'].fill(dataset=dataset, flav=flavLabel,
                                          pt=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].pt)))
            output['photon_eta_hist'].fill(dataset=dataset, flav=flavLabel,
                                           eta=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].eta)))
            output['photon_phi_hist'].fill(dataset=dataset, flav=flavLabel,
                                           phi=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].phi)))
            output['photon_reliso_all_hist'].fill(dataset=dataset, flav=flavLabel,
                                                  reliso=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].pfRelIso03_all)))
            output['photon_reliso_chg_hist'].fill(dataset=dataset, flav=flavLabel,
                                                  reliso=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].pfRelIso03_chg)))
            output['photon_sieie_hist'].fill(dataset=dataset, flav=flavLabel,
                                             sieie=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].sieie)))
            output['photon_r9_hist'].fill(dataset=dataset, flav=flavLabel,
                                          r9=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].r9)))
            output['photon_hoe_hist'].fill(dataset=dataset, flav=flavLabel,
                                           hoe=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].hoe)))
            
            output['deltaR_photon_lepton_hist'].fill(dataset=dataset, flav=flavLabel,
                                                     deltar=ak.to_numpy(ak.flatten(tightPhoMuDR[eventSelection])))
            output['deltaR_photon_jet_hist'].fill(dataset=dataset, flav=flavLabel,
                                                  deltar=ak.to_numpy(ak.flatten(tightPhoJetDR[eventSelection])))
            
            output['photon_mvaid_hist'].fill(dataset=dataset, flav=flavLabel,
                                             mvaid=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].mvaID)))
        
            output['photon_genPartFlav_hist'].fill(dataset=dataset,
                                                   flav=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].genPartFlav)))
        
        ######## fill histograms: Data ########
        if self.isMC==0:
            output['photon_pt_hist'].fill(pt=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].pt)))
            output['photon_eta_hist'].fill(eta=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].eta)))
            output['photon_phi_hist'].fill(phi=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].phi)))
            output['photon_reliso_all_hist'].fill(reliso=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].pfRelIso03_all)))
            output['photon_reliso_chg_hist'].fill(reliso=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].pfRelIso03_chg)))
            output['photon_sieie_hist'].fill(sieie=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].sieie)))
            output['photon_r9_hist'].fill(r9=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].r9)))
            output['photon_hoe_hist'].fill(hoe=ak.to_numpy(ak.flatten(tightPhotons[eventSelection].hoe)))
            
            output['deltaR_photon_lepton_hist'].fill(deltar=ak.to_numpy(ak.flatten(tightPhoMuDR[eventSelection])))
            output['deltaR_photon_jet_hist'].fill(deltar=ak.to_numpy(ak.flatten(tightPhoJetDR[eventSelection])))
            
        
        ######## fill column accumulators ########
        output['photon_pt'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.pt[eventSelection])))
        output['photon_eta'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.eta[eventSelection])))
        output['photon_phi'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.phi[eventSelection])))
        output['photon_pfRelIso03_all'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.pfRelIso03_all[eventSelection])))
        output['photon_pfRelIso03_chg'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.pfRelIso03_chg[eventSelection])))
        output['photon_sieie'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.sieie[eventSelection])))
        output['photon_r9'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.r9[eventSelection])))
        output['photon_hoe'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.hoe[eventSelection])))
        
        output['deltaR_photon_lepton'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhoMuDR[eventSelection])))
        output['deltaR_photon_jet'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhoJetDR[eventSelection])))
        
        output['photon_mvaid'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.mvaID[eventSelection])))
        
        if self.isMC:
            output['photon_genPartFlav'] = processor.column_accumulator(ak.to_numpy(ak.flatten(tightPhotons.genPartFlav[eventSelection])))
            
            if dataset=='TTGamma':
                output['file'] = processor.column_accumulator(np.ones_like(ak.to_numpy(ak.flatten(tightPhotons.pt[eventSelection]))))
            elif dataset=='TTbar':
                output['file'] = processor.column_accumulator(np.zeros_like(ak.to_numpy(ak.flatten(tightPhotons.pt[eventSelection]))))
            else:
                output['file'] = processor.column_accumulator(2*np.ones_like(ak.to_numpy(ak.flatten(tightPhotons.pt[eventSelection]))))
        
        
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [5]:
# Define files to run over
skimDir="/udrive/staff/dnoonan/Skims"

fileset = {"TTGamma":[f"{skimDir}/TTGamma_SingleLept_2016_skim.root"],
           "TTbar":[f"{skimDir}/TTbarPowheg_Semilept_2016_skim_1of10.root",
                    f"{skimDir}/TTbarPowheg_Semilept_2016_skim_2of10.root",
                    f"{skimDir}/TTbarPowheg_Semilept_2016_skim_3of10.root",
                    f"{skimDir}/TTbarPowheg_Semilept_2016_skim_4of10.root",
                    f"{skimDir}/TTbarPowheg_Semilept_2016_skim_5of10.root"],
           "WGamma":[f"{skimDir}/WGamma_2016_skim.root"],
           "Z+jets":[f'{skimDir}/DYjetsM50_ext2_2016_skim_1of10.root'],
           "W+3jets":[f"{skimDir}/W3jets_2016_skim.root"],
           "W+4jets":[f"{skimDir}/W4jets_2016_skim.root"],
          }
filesetData = {"DataMu":[f"{skimDir}/Data_SingleMu_b_2016_skim_1of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_2of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_3of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_4of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_5of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_6of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_7of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_8of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_9of10.root",
                         f"{skimDir}/Data_SingleMu_b_2016_skim_10of10.root"]
              }

In [6]:
np.warnings.filterwarnings('ignore')

# The NanoAODSchema needs to be adjusted, to remove cross references to FSRPhotons
class SkimmedSchema(NanoAODSchema):
    def __init__(self, base_form):
        base_form["contents"].pop("Muon_fsrPhotonIdx", None)
        super().__init__(base_form)

In [7]:
# Run Coffea code using uproot

## Monte Carlo
outputMC = processor.run_uproot_job(
    fileset,
    "Events",
    PhotonSelector(),
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4},
    chunksize=1000000,
    #maxchunks=3,
)

Preprocessing:   0%|          | 0/10 [00:00<?, ?file/s]

Processing:   0%|          | 0/35 [00:00<?, ?chunk/s]

In [8]:
## Real data

outputData = processor.run_uproot_job(
    filesetData,
    "Events",
    PhotonSelector(isMC=False),
    processor.futures_executor,
    executor_args={"schema": SkimmedSchema,'workers': 4},
    chunksize=1000000,
    #maxchunks=3,
)

Preprocessing:   0%|          | 0/10 [00:00<?, ?file/s]

Processing:   0%|          | 0/78 [00:00<?, ?chunk/s]

## Luminosity scaling

In [9]:
# get number of events in each dataset
nEvents = {}
for d in fileset:
    if not d in nEvents:
        nEvents[d] = 0
    for fName in fileset[d]:
        with uproot.open(fName)['hEvents'] as hEvents:
            nEvents[d] += hEvents.values()[0] + hEvents.values()[2]

print(nEvents)

{'TTGamma': 11005200.0, 'TTbar': 51018200.0, 'WGamma': 6103817.0, 'Z+jets': 8920292.0, 'W+3jets': 19798117.0, 'W+4jets': 9116657.0}


In [10]:
# cross-section
cx = {'TTGamma':7.509,
     'TTbar': 380.095,
     'WGamma':489,
     'Z+jets':6077.22,
     'W+3jets':1165.8108,
     'W+4jets':592.9176}

# weights
lumi_weight = {}
for keyName in fileset:
    lumi_weight[keyName] = (cx[keyName]*5780.)/nEvents[keyName]

print(lumi_weight)

{'TTGamma': 0.003943773852360702, 'TTbar': 0.043062066086220215, 'WGamma': 0.46305778826593263, 'Z+jets': 3.937800645987822, 'W+3jets': 0.34035491476285346, 'W+4jets': 0.37591232487961324}


In [11]:
for key, obj in outputMC.items():
    if (isinstance(obj, hist.Hist)):
        obj.scale(lumi_weight, axis="dataset")

## Save outputs

In [12]:
save(outputMC,'outputMC.coffea')
save(outputData,'outputData.coffea')